# Capita

Technical Challenge for Data Science Candidates

Question 2: Exploratory data analysis to show how the population progressed by regions and age groups. Part 1. Data preparation.

In [125]:
import numpy as np
import pandas as pd
import math
import json

from pyjstat import pyjstat
from os import path

import scipy.stats as st
import statsmodels as sm
import statsmodels.api as smi

import matplotlib
from cycler import cycler
import matplotlib.pyplot as plt

# this is the local Utility module
from cpta import Utility

%load_ext autoreload
%autoreload 2

pd.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'0.24.2'

In [27]:
# If you turn this feature on, you can display each result as it happens.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [28]:
fl2 = 'stats.pickle'
path.exists(fl2)

True

In [29]:
# There's a future warning - minor change made in pyjstat
if not path.exists(fl2):
    fl1 = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_31_1.jsonstat.json'
    dataset = pyjstat.Dataset.read(fl1)
    df = dataset.write('dataframe')
    df.to_pickle(fl2)
else:
    df = pd.read_pickle(fl2)

In [108]:
## Categories can be simpler
df.date = df.date.astype('int32')
df.info()
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 6 columns):
date         22800 non-null int32
geography    22800 non-null category
sex          22800 non-null category
age          22800 non-null category
measures     22800 non-null category
value        20358 non-null float64
dtypes: category(4), float64(1), int32(1)
memory usage: 357.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 6 columns):
date         22800 non-null int32
geography    22800 non-null category
sex          22800 non-null category
age          22800 non-null category
measures     22800 non-null category
value        20358 non-null float64
dtypes: category(4), float64(1), int32(1)
memory usage: 357.5 KB


In [109]:
# This utility has a simplify0 method that applies question 1 manipulations to any age group and year
i0 = Utility.instance()

In [110]:
# It splits up into these iterations.
ages = set(df.age.values)
dates = set(df.date)

In [111]:
# Check the expected length.

df1 = i0.simplify0(df)
df1
len(ages)
len(dates)
len(ages) * len(dates) * len(set(df1.geography))

,date,geography,age,Male,Female,Total
0,2018,England,All ages,27668000.0,28309300.0,55977200.0
1,2018,Northern Ireland,All ages,926200.0,955400.0,1881600.0
2,2018,Scotland,All ages,2648800.0,2789300.0,5438100.0
3,2018,UK,All ages,32790300.0,33645300.0,66435500.0
4,2018,Wales,All ages,1547300.0,1591300.0,3138600.0


25

38

4750

In [112]:
# Form parameter pairs and iterate through them all.
# This is very time-consuming.

t0 = [ zip([x] * len(ages), ages) for x in dates ]

def step0():
    """
    This provides an iterator across the parameters in t0 and returns the result of an invocation of simplify0
    """
    for x in t0:
        for dt0, age0 in x:
            yield i0.simplify0(df, dt0=dt0, age0=age0)

t1 = [ x for x in step0() ]
len(t1)

950

In [199]:
df2 = pd.concat(t1).reset_index()
df2.drop(columns='index', axis=1, inplace=True)
df2.info()

## Reduce numbers to millions
df2.Male = df2.Male / 1000000
df2.Female = df2.Female / 1000000
df2.Total = df2.Total / 1000000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4750 entries, 0 to 4749
Data columns (total 6 columns):
date         4750 non-null int64
geography    4750 non-null object
age          4750 non-null object
Male         4343 non-null float64
Female       4343 non-null float64
Total        4343 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 222.7+ KB


In [200]:
df2.to_pickle('stats1.pickle')

In [201]:
ages

{'Aged 0 - 15',
 'Aged 1 - 4 years',
 'Aged 10 - 14 years',
 'Aged 15 - 19 years',
 'Aged 16 - 59/64',
 'Aged 16 - 64',
 'Aged 18 - 24',
 'Aged 20 - 24 years',
 'Aged 25 - 29 years',
 'Aged 30 - 34 years',
 'Aged 35 - 39 years',
 'Aged 40 - 44 years',
 'Aged 45 - 49 years',
 'Aged 5 - 9 years',
 'Aged 50 - 54 years',
 'Aged 55 - 59 years',
 'Aged 60 - 64 years',
 'Aged 65 - 69 years',
 'Aged 65 and over',
 'Aged 70 - 74 years',
 'Aged 75 - 79 years',
 'Aged 80 - 84 years',
 'Aged 85 and over',
 'Aged under 1 year',
 'All ages'}

## Making use of Excel's Pivot Table Feature

Excel's Pivot Chart is a useful visualisation tool. Produce some output CSV files to make use of it. (Note what is done here can be done in Excel using filters on the pivot table or manipulating the data. It is easier to do it here.)

To make use of the column sum and row sum feature, let's remove the sum figures in the data. 

 - By country: remove UK
 - By age group: remove All ages
 
A point to note is that age groups appear in two types:

 - By band: 'Aged 0 - 15', 'Aged 16 - 59/64', 'Aged 65 and over', 'All ages'
 - By twientieth: ie. 5 years at a time.

In [202]:
## Use Excel for some pivot charts

# Use this to generate filenames
i0._seq0 = 0
fmt = lambda t0: "cache/bak/cpta-{:02d}.csv".format(t0)
flnmr0 = lambda : next(i0.seq0(fmt=fmt))

In [203]:
# Write out the unadulterated data
df2.to_csv(flnmr0(), index=False)

In [204]:
## Use column sum feature of Excel's pivot table to show proportions by country
# remove all UK 

(df2[ df2.geography != 'UK' ]).to_csv(flnmr0(), index=False)

In [205]:
## By age-bands
agesB = set([ 'Aged 0 - 15', 'Aged 16 - 59/64', 'Aged 65 and over' ])

In [206]:
( df2[ df2.age.isin(agesB) ] ).to_csv(flnmr0(), index=False)

In [207]:
ages20 = ages.difference(agesB.union(set(['All ages'])))
ages20

{'Aged 1 - 4 years',
 'Aged 10 - 14 years',
 'Aged 15 - 19 years',
 'Aged 16 - 64',
 'Aged 18 - 24',
 'Aged 20 - 24 years',
 'Aged 25 - 29 years',
 'Aged 30 - 34 years',
 'Aged 35 - 39 years',
 'Aged 40 - 44 years',
 'Aged 45 - 49 years',
 'Aged 5 - 9 years',
 'Aged 50 - 54 years',
 'Aged 55 - 59 years',
 'Aged 60 - 64 years',
 'Aged 65 - 69 years',
 'Aged 70 - 74 years',
 'Aged 75 - 79 years',
 'Aged 80 - 84 years',
 'Aged 85 and over',
 'Aged under 1 year'}

In [208]:
## By 20ths

( df2[ df2.age.isin(ages20) ] ).to_csv(flnmr0(), index=False)